<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Basic-Functions" data-toc-modified-id="Basic-Functions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Basic Functions</a></span></li><li><span><a href="#Offline-Rerun" data-toc-modified-id="Offline-Rerun-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Offline Rerun</a></span><ul class="toc-item"><li><span><a href="#Status-==-1" data-toc-modified-id="Status-==-1-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Status == 1</a></span></li></ul></li><li><span><a href="#Sampling" data-toc-modified-id="Sampling-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Sampling</a></span><ul class="toc-item"><li><span><a href="#ID" data-toc-modified-id="ID-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>ID</a></span><ul class="toc-item"><li><span><a href="#Get-Preferred-Shops" data-toc-modified-id="Get-Preferred-Shops-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Get Preferred Shops</a></span></li><li><span><a href="#Get-Excluded-Items" data-toc-modified-id="Get-Excluded-Items-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>Get Excluded Items</a></span></li><li><span><a href="#Get-Sample-Size" data-toc-modified-id="Get-Sample-Size-3.1.3"><span class="toc-item-num">3.1.3&nbsp;&nbsp;</span>Get Sample Size</a></span></li><li><span><a href="#Processing" data-toc-modified-id="Processing-3.1.4"><span class="toc-item-num">3.1.4&nbsp;&nbsp;</span>Processing</a></span></li></ul></li></ul></li></ul></div>

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import os
import numpy as np
import pandas as pd
import datetime
import time
import math

import numpy as np
import pandas as pd

import langdetect as lgd
from googletrans import Translator

import warnings
warnings.filterwarnings('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

---

In [14]:
offlineDataFolder = '/Users/xu.zhu/Desktop/Data/Translaltion/OfflineRerun'

In [15]:
sampleSizeDict = {}
sampleSizeDict['TH'] = 5000
sampleSizeDict['ID'] = 35000

sampleSizeDict

{'TH': 5000, 'ID': 35000}

In [16]:
region = input("Please Entry Region:")
if region.lower() == 'th':
    sampleSize = sampleSizeDict['TH']
    
    cat_dict = {}
    cat_dict['Mobile & Gadgets'] = 0.2
    cat_dict["Women Clothes"] = 0.2 
    cat_dict["Baby & Toys"] = 0.1
    cat_dict["Women Shoes"] = 0.1 
    cat_dict["Beauty & Personal Care"] = 0.1
    cat_dict["Home & Living"] = 0.1
    cat_dict["Watches & Glasses"] = 0.1
    cat_dict["Bags"] = 0.1
    
elif region.lower() == 'id':
    cat_dict = {}
    cat_dict["Home & Living"] = 0.2 
    cat_dict["Fashion Accessories"] = 0.15
    cat_dict["Mobile & Accessories"] = 0.15
    cat_dict["Hobby & Collection"] = 0.10
    cat_dict["Beauty"] = 0.10
    cat_dict["Women Clothes"] = 0.08
    cat_dict["Baby & Kids Fashion"] = 0.08
    cat_dict["Mom & Baby"] = 0.05
    cat_dict["Electronics"] = 0.05
    cat_dict["Sports & Outdoor"] = 0.04

else:
    pass

print(cat_dict)

Please Entry Region:id
{'Home & Living': 0.2, 'Fashion Accessories': 0.15, 'Mobile & Accessories': 0.15, 'Hobby & Collection': 0.1, 'Beauty': 0.1, 'Women Clothes': 0.08, 'Baby & Kids Fashion': 0.08, 'Mom & Baby': 0.05, 'Electronics': 0.05, 'Sports & Outdoor': 0.04}


## Basic Functions

In [19]:
def gsheet_get_credentials(scopes, folder_path):
    creds = None
    crendentialPath = os.path.join(folder_path, 'Gsheet_py3_credentials.json')
    tokenPath = os.path.join(folder_path, 'Gsheet_py3_token.pickle')

    if os.path.exists(tokenPath):
        with open(tokenPath, 'rb') as token:
            creds = pickle.load(token)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(crendentialPath, scopes)
            creds = flow.run_local_server()
        with open(tokenPath, 'wb') as token:
            pickle.dump(creds, token)

    return creds


def gsheet_read_data(gsheetCreds, gsheetID, gsheetReadRange,
                     outputOption = 'DataFrame', printInfo = True):
    gsheetService = build('sheets', 'v4', credentials = gsheetCreds)
    sheet = gsheetService.spreadsheets()
    request = sheet.values().get(spreadsheetId = gsheetID, range = gsheetReadRange)
    response = request.execute()
    values = response['values']

    if outputOption.lower() == 'list':
        # result = [value[0] for value in values if value != []]
        result = []
        for value in values:
            if value == []:
                continue
            else:
                result.append(value[0])

    elif outputOption.lower() == 'string':
        pyVersion = int(sys.version_info[0])
        if pyVersion == 2:
            resultTemp = [str(value[0]).encode('utf-8') for value in values if value != []]
        else:
            resultTemp = [str(value[0]) for value in values if value != []]
        result = "(" + ",".join(resultTemp) + ")"

    else:
        cols = values.pop(0)
        result = pd.DataFrame(data = values, columns = cols)

    if printInfo == True:
        gsheetURL = 'https://docs.google.com/spreadsheets/d/{0}'.format(gsheetID)
        print("----- AUTO READ")
        print("* Read Data From: {0}".format(gsheetURL))
        print("* Read Range: {0}".format(gsheetReadRange))
        print("* Output Option: {0}".format(outputOption.lower()))
    else:
        pass

    return result


def gdrive_get_credentials(scopes, folder_path):
    creds = None
    crendential_path = os.path.join(folder_path, 'Gdrive_py3_credentials.json')
    token_path = os.path.join(folder_path, 'Gdrive_py3_token.pickle')

    if os.path.exists(token_path):
        with open(token_path, 'rb') as token:
            creds = pickle.load(token)
    
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(crendential_path, scopes)
            creds = flow.run_local_server(port=0)

        # Save the credentials for the next run
        with open(token_path, 'wb') as token:
            pickle.dump(creds, token)

    return creds


def gdrive_download(gdriveCreds, gdriveFolderID, downloadFolderPath, gdriveFileMatchKeys,
                    printInfo = True):
    matchedFiles = []
    downloadedFiles = []
    gdriveURL = 'https://drive.google.com/drive/u/0/folders/{0}'.format(gdriveFolderID)

    gdriveService = build('drive', 'v3', credentials = gdriveCreds)
    gdriveSearchQuery = "'{0}' in parents".format(gdriveFolderID)
    listedResponse = gdriveService.files().list(q = gdriveSearchQuery).execute()

    listedFiles = listedResponse['files']
    for gdriveFile in listedFiles:
        gdirveFileID = gdriveFile['id']
        gdriveFileName = gdriveFile['name']

        for _ in gdriveFileMatchKeys:
            if "translated" in gdriveFileName and _ in gdriveFileName: # 
                matchedFiles.append(gdriveFileName)
                request = gdriveService.files().get_media(fileId = gdirveFileID)
                
                # Method I
                with open(os.path.join(downloadFolderPath, gdriveFileName), 'wb') as f:
                    downloader = MediaIoBaseDownload(f, request)
                    done = False
                    while done is False:
                        status, done = downloader.next_chunk()
                downloadedFiles.append(gdriveFileName)

                # Method II
                # fh = io.BytesIO(request.execute())
                # with io.open(os.path.join(downloadFolderPath, gdriveFileName), 'wb') as f:
                #     fh.seek(0)
                #     f.write(fh.read())
                # print("Downloaded {0}".format(gdriveFileName))
            else:
                pass

    if printInfo == True:
        print("----- AUTO DOWNLOAD")
        print("* Downloade Gdrive Files From: {0}".format(gdriveURL))
        print("* Matching Keys: translated & {0}".format(gdriveFileMatchKeys))
        print("* Matched {0} Files:\n\t{1}".format(len(matchedFiles), matchedFiles))
        print("* Downloaded {0} Files:\n\t{1}".format(len(downloadedFiles), downloadedFiles))
    else:
        pass

    return downloadedFiles


def gdrive_upload(gdriveCreds, gdriveFolderID, allFilePaths, 
                  newFolderOption = True, printInfo = True, 
                  **kwargs):
    uploadedFiles = []
    gdriveService = build('drive', 'v3', credentials = gdriveCreds)
    gdriveURL = 'https://drive.google.com/drive/u/0/folders/{0}'.format(gdriveFolderID)

    if newFolderOption == True:
        if 'newFolderName' in kwargs.keys():
            newFolderName = kwargs['newFolderName']
        else:
            newFolderName = kwargs[list(kwargs.keys())[0]]

        folderMetaData = {'name': str(newFolderName), 
                          'mimeType': 'application/vnd.google-apps.folder', 
                          'parents': [gdriveFolderID]}

        newFolderObject = gdriveService.files().create(body = folderMetaData,
                                                       fields = 'id').execute()
        folderID = newFolderObject['id']
    else:
        folderID = gdriveFolderID

    for filePath in allFilePaths:
        uploadedFileName = filePath.split('/')[-1]
        fileMetaData = {"name": uploadedFileName, "parents": [folderID]}

        # Here only considering csv and xlsx 2 formats
        # More MIME Types Refer: https://developers.google.com/drive/api/v3/ref-export-formats
        uploadedFileFormat = uploadedFileName.split('.')[-1]
        if uploadedFileFormat == 'csv':
            uploadedFileType = 'text/csv'
        else:
            uploadedFileType = 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
        
        media = MediaFileUpload(filePath, mimetype = uploadedFileType)
        file = gdriveService.files().create(body = fileMetaData,
                                            media_body = media,
                                            fields = 'id').execute()
        
        uploadedFiles.append(uploadedFileName)
    
    if printInfo == True:
        print("----- AUTO UPLOAD")
        print("* Upload Files To Gdrive: {0}".format(gdriveURL))
        print("* Uploaded {0} Files:\n\t{1}".format(len(uploadedFiles), uploadedFiles))
    else:
        pass

    return uploadedFiles

## Offline Rerun

In [17]:
offlineFile = os.path.join(offlineDataFolder, '2020-08-19_ID_Translation_Raw-translated-below.xlsx')
raw_df = pd.read_excel(offlineFile, encoding = 'utf-8-sig')
raw_df.shape

raw_df.head(3)

(40834, 8)

,shopid,itemid,main_name,product_name,product_url,translation,score,flag
0,11818526,4146670623,Beauty & Care,MAX Flakes Slices Mirror Sparkly Stars Laser N...,https://shopee.co.id/a-i.11818526.4146670623,Hiasan Kuku Sequin Laser Motif Bintang Berkila...,0.465929,False
1,28079462,3946788157,Home & Living,Auto Darkening Solar Welders Welding Helmet Mask,https://shopee.co.id/a-i.28079462.3946788157,Helm Pelindung Wajah Dengan Kaca Gelap Otomati...,0.497638,False
2,28079462,3746361132,Automotive,12V 24V 300db Super Loud Four/ 4 Trumpet Air H...,https://shopee.co.id/a-i.28079462.3746361132,Klakson Angin Super Keras 4/4 Terompet 12v 24v...,0.748091,False


In [18]:
set(raw_df.flag.to_list())

{False}

### Status == 1

In [10]:
raw_df = raw_df[raw_df['status'] == 1]
raw_df.shape

cols = raw_df.columns.to_list()
cols

(9905, 10)

['shopid',
 'itemid',
 'Username',
 'Category',
 'product_name',
 'product_url',
 'status',
 'translation',
 'score',
 'flag']

## Sampling

In [22]:
import pickle
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload

### ID

In [24]:
today = datetime.datetime.today().strftime('%F')
d1 = (datetime.datetime.today() - datetime.timedelta(days = 1)).strftime("%Y-%m-%d")
d2 = (datetime.datetime.today() - datetime.timedelta(days = 2)).strftime("%Y-%m-%d")
d3 = (datetime.datetime.today() - datetime.timedelta(days = 3)).strftime("%Y-%m-%d")

downloadFolderPath = '/Users/xu.zhu/Desktop/Data/Translaltion/OfflineRerun'
outputFolderPath = '/Users/xu.zhu/Desktop/Data/Translaltion/OfflineRerun'
credentialFolder = '/Users/xu.zhu/Desktop/Data/Keys/GoogleAPI/credentials'

In [25]:
gsheetScopes = 'https://www.googleapis.com/auth/spreadsheets'
gsheetCreds = gsheet_get_credentials(gsheetScopes, credentialFolder)

gdriveScopes = ['https://www.googleapis.com/auth/drive']
gdriveCreds = gdrive_get_credentials(gdriveScopes, credentialFolder)

#### Get Preferred Shops

In [30]:
gsheetID_PreferredShops = '1JlXpnTj1WDfcIrWUPc6VJTccc9ZehS9Gjw1ob0XgFME' 
gsheetRange_PreferredShops = 'Focus shop!A1:D'
preferredShops_df = gsheet_read_data(gsheetCreds, gsheetID_PreferredShops, gsheetRange_PreferredShops)

preferredShops_df['shopid'] = preferredShops_df['shopid'].astype(int)
preferredShops = list(set(preferredShops_df['shopid'].to_list()))

preferredShops_df.shape
preferredShops_df.head(2)

----- AUTO READ
* Read Data From: https://docs.google.com/spreadsheets/d/1JlXpnTj1WDfcIrWUPc6VJTccc9ZehS9Gjw1ob0XgFME
* Read Range: Focus shop!A1:D
* Output Option: dataframe


(89, 4)

,S/N,shopid,seller username,seller type/reason to prioritize
0,1,76523971,o.two.o.id,JBP
1,2,77707111,focallureshop.id,JBP


#### Get Excluded Items

In [29]:
gsheetID_ExcludedItems = '1tMhexIR3_WnG9USuv1Tz4qTEhelu984cZnB6ez20MYI'
gsheetRange_ExcludedItems = 'abnormal_item!A:A'
excludedItems_df = gsheet_read_data(gsheetCreds, gsheetID_ExcludedItems, gsheetRange_ExcludedItems)

excludedItems_df['itemid'] = excludedItems_df['itemid'].astype(int)

excludedItems_df.shape
excludedItems_df.head(2)

----- AUTO READ
* Read Data From: https://docs.google.com/spreadsheets/d/1tMhexIR3_WnG9USuv1Tz4qTEhelu984cZnB6ez20MYI
* Read Range: abnormal_item!A:A
* Output Option: dataframe


(500000, 1)

,itemid
0,4802071490
1,4401472164


#### Get Sample Size

In [43]:
gsheetID_SampleSize = '1YZlRGsqD3lTJgPRlN7PSTQiFmVdXuN15ZeCp0f18stw'
gsheetRange_SampleSize = 'Human_translation_capacity!A1:B'
sampleSize_df = gsheet_read_data(gsheetCreds, gsheetID_SampleSize, gsheetRange_SampleSize)
sampleSize_df['capacity'] = sampleSize_df['capacity'].astype(int)

sampleSize = int(sampleSize_df[sampleSize_df['country'] == 'ID']['capacity'])
sampleSize_df
sampleSize

----- AUTO READ
* Read Data From: https://docs.google.com/spreadsheets/d/1YZlRGsqD3lTJgPRlN7PSTQiFmVdXuN15ZeCp0f18stw
* Read Range: Human_translation_capacity!A1:B
* Output Option: dataframe


,country,capacity
0,ID,35000
1,TH,5000


35000

#### Processing

In [46]:
cat_dict = {}
cat_dict["Home & Living"] = 0.2 
cat_dict["Fashion Accessories"] = 0.15
cat_dict["Mobile & Accessories"] = 0.15
cat_dict["Hobby & Collection"] = 0.10
cat_dict["Beauty"] = 0.10
cat_dict["Women Clothes"] = 0.08
cat_dict["Baby & Kids Fashion"] = 0.08
cat_dict["Mom & Baby"] = 0.05
cat_dict["Electronics"] = 0.05
cat_dict["Sports & Outdoor"] = 0.04

In [45]:
raw_df['preferred_shop'] = raw_df['shopid'].apply(lambda x: "Y" if x in preferredShops else "N")
raw_df

,shopid,itemid,main_name,product_name,product_url,translation,score,flag,preferred_shop
0,11818526,4146670623,Beauty & Care,MAX Flakes Slices Mirror Sparkly Stars Laser N...,https://shopee.co.id/a-i.11818526.4146670623,Hiasan Kuku Sequin Laser Motif Bintang Berkila...,0.465929,False,N
1,28079462,3946788157,Home & Living,Auto Darkening Solar Welders Welding Helmet Mask,https://shopee.co.id/a-i.28079462.3946788157,Helm Pelindung Wajah Dengan Kaca Gelap Otomati...,0.497638,False,N
2,28079462,3746361132,Automotive,12V 24V 300db Super Loud Four/ 4 Trumpet Air H...,https://shopee.co.id/a-i.28079462.3746361132,Klakson Angin Super Keras 4/4 Terompet 12v 24v...,0.748091,False,N
3,28079462,3158311528,Automotive,Fleece Lined Urban Scooter Electric Bike Leg C...,https://shopee.co.id/a-i.28079462.3158311528,Manset Kaki Dengan Pelindung Lutut Bahan Fleec...,0.460026,False,N
4,28079462,7746472434,Automotive,Electric Pressure Washer Power Washer Cordless...,https://shopee.co.id/a-i.28079462.7746472434,Mesin Pencuci Mobil Elektrik Portable Tanpa Ka...,0.482208,False,N
...,...,...,...,...,...,...,...,...,...
40829,263774352,6946186074,Home & Living,Health Care Bed Sores Pregnancy Sciatica Offic...,https://shopee.co.id/a-i.263774352.6946186074,Bantal Duduk Bahan Memory Foam Untuk Pereda Sa...,0.460216,False,N
40830,263774352,5146615010,Home & Living,Home Adjustable Flower Window Garden Watering ...,https://shopee.co.id/a-i.263774352.5146615010,Nozzle Selang Air Adjustable Untuk Jendela/Tam...,0.497555,False,N
40831,263774352,5746988151,Home & Living,Equipment Gift Home Outdoor Spur Wire Embedder,https://shopee.co.id/a-i.263774352.5746988151,Spurder Bahan Stainless Steel Untuk Rumah\n,0.442754,False,N
40832,263774352,5946160838,Home & Living,Home Transparent Storage Sealed Compression Fo...,https://shopee.co.id/a-i.263774352.5946160838,Tas Penyimpanan Pakaian Model Gantung Dengan S...,0.462040,False,N


In [50]:
prefer_df = raw_df[raw_df['preferred_shop'] == "Y"]
remain_df = raw_df[raw_df['preferred_shop'] != "Y"]

preferredNum = prefer_df.shape[0]
nonPreferredNum = remain_df.shape[0]

In [51]:
remainNum = sampleSize - preferredNum
remainNum

35000

In [55]:
test = remain_df.sample(n = 1)

In [58]:
pd.merge(remain_df, test, how = 'left', indicator = True)

,shopid,itemid,main_name,product_name,product_url,translation,score,flag,preferred_shop,_merge
0,11818526,4146670623,Beauty & Care,MAX Flakes Slices Mirror Sparkly Stars Laser N...,https://shopee.co.id/a-i.11818526.4146670623,Hiasan Kuku Sequin Laser Motif Bintang Berkila...,0.465929,False,N,left_only
1,28079462,3946788157,Home & Living,Auto Darkening Solar Welders Welding Helmet Mask,https://shopee.co.id/a-i.28079462.3946788157,Helm Pelindung Wajah Dengan Kaca Gelap Otomati...,0.497638,False,N,left_only
2,28079462,3746361132,Automotive,12V 24V 300db Super Loud Four/ 4 Trumpet Air H...,https://shopee.co.id/a-i.28079462.3746361132,Klakson Angin Super Keras 4/4 Terompet 12v 24v...,0.748091,False,N,left_only
3,28079462,3158311528,Automotive,Fleece Lined Urban Scooter Electric Bike Leg C...,https://shopee.co.id/a-i.28079462.3158311528,Manset Kaki Dengan Pelindung Lutut Bahan Fleec...,0.460026,False,N,left_only
4,28079462,7746472434,Automotive,Electric Pressure Washer Power Washer Cordless...,https://shopee.co.id/a-i.28079462.7746472434,Mesin Pencuci Mobil Elektrik Portable Tanpa Ka...,0.482208,False,N,left_only
...,...,...,...,...,...,...,...,...,...,...
40650,263774352,6946186074,Home & Living,Health Care Bed Sores Pregnancy Sciatica Offic...,https://shopee.co.id/a-i.263774352.6946186074,Bantal Duduk Bahan Memory Foam Untuk Pereda Sa...,0.460216,False,N,left_only
40651,263774352,5146615010,Home & Living,Home Adjustable Flower Window Garden Watering ...,https://shopee.co.id/a-i.263774352.5146615010,Nozzle Selang Air Adjustable Untuk Jendela/Tam...,0.497555,False,N,left_only
40652,263774352,5746988151,Home & Living,Equipment Gift Home Outdoor Spur Wire Embedder,https://shopee.co.id/a-i.263774352.5746988151,Spurder Bahan Stainless Steel Untuk Rumah\n,0.442754,False,N,left_only
40653,263774352,5946160838,Home & Living,Home Transparent Storage Sealed Compression Fo...,https://shopee.co.id/a-i.263774352.5946160838,Tas Penyimpanan Pakaian Model Gantung Dengan S...,0.462040,False,N,left_only
